In [ ]:
# hide
# default_exp utils.cli
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# cli tools

> CLI tools

In [ ]:
#export
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.merge import *
from nbdev.export2html import *
from nbdev.clean import *
from nbdev.test import *
from fastcore.script import *

In [ ]:
#for tests
import os

In [ ]:
# export
@call_parse
def dsblocks_install_git_hooks():
    "Install git hooks to clean/trust notebooks automatically"
    try: path = get_config().config_file.parent
    except: path = Path.cwd()
    hook_path = path/'.git'/'hooks'
    fn = hook_path/'pre-commit'
    hook_path.mkdir(parents=True, exist_ok=True)
    env_name = os.environ['CONDA_DEFAULT_ENV']
    fn.write_text(f"""#!/bin/sh

echo "\n==================== pre-commit hook ===================="

# Export conda environment to yaml file
conda env export > {env_name}.yml

# Check if new environment file is different from original 
git diff --exit-code --quiet {env_name}.yml 

# If new environment file is different, commit it
if [ $? -eq 0 ]
then
    echo "Conda environment not changed. No additional commit."
else
    echo "Conda environment changed. Commiting new {env_name}.yml"
    git add {env_name}.yml
    git commit -m "Updating conda environment {env_name}"
    exit 1
fi
""")
    
    os.chmod (fn, stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR | stat.S_IRGRP | stat.S_IWGRP | stat.S_IXGRP | stat.S_IROTH |  stat.S_IXOTH)
    
    fn = hook_path/'post-merge'
    hook_path.mkdir(parents=True, exist_ok=True)
    
    fn.write_text(f"""#!/bin/sh

echo "\n==================== post-merge hook ===================="

changed_files="$(git diff-tree -r --name-only --no-commit-id ORIG_HEAD HEAD)"

check_run() {{
    echo "$changed_files" | grep --quiet "$1" && eval "$2"
}}

echo "Have to update the conda environment"
check_run {env_name}.yml "conda env update --file {env_name}.yml"
""")
    
    os.chmod (fn, stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR | stat.S_IRGRP | stat.S_IWGRP | stat.S_IXGRP | stat.S_IROTH |  stat.S_IXOTH)

### Usage example

In [ ]:
# export tests.utils.test_cli  
def test_dsblocks_install_git_hooks ():
    if os.path.exists('.git/hooks/pre-push'):
        os.remove ('.git/hooks/pre-push')

    if os.path.exists('.git/hooks/post-merge'):
        os.remove ('.git/hooks/post-merge')
    
    dsblocks_install_git_hooks ()
    assert {'post-merge', 'pre-commit'}.issubset(os.listdir ('.git/hooks'))
    with open('.git/hooks/pre-commit', 'rt') as f: txt = f.read ()
    assert txt.startswith ('#!/bin/sh\n\necho')
    assert txt.endswith ('fi\n')

    with open('.git/hooks/post-merge', 'rt') as f: txt = f.read ()
    assert txt.startswith ('#!/bin/sh\n\necho')
    env_name = os.environ['CONDA_DEFAULT_ENV']
    assert txt.endswith (f'le {env_name}.yml"\n')
    
    os.remove ('.git/hooks/pre-commit')
    os.remove ('.git/hooks/post-merge')

In [ ]:
tst.run (test_dsblocks_install_git_hooks, tag='dummy')

running test_dsblocks_install_git_hooks
